In [3]:
import numpy as np
from numpy import genfromtxt as csv_load
import sys
import csv
import os


def checkTOs(v):
    tos = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 0 and v[s+1] == 1):
            tos.append(count)
    return tos

def checkHSs(v):
    hss = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 1 and v[s+1] == 0):
            hss.append(count)
    return hss

def evaluate(ref_tr, pred_tr, tolerance):
    positives = len(ref_tr)
    tp = 0;
    fp = 0;
    fn =0;
    diffs = []
    for p_tr in pred_tr:
        found = False
        for r_tr in ref_tr:
            diff = abs(r_tr-p_tr)
            if (diff <= tolerance):
                #print("r_tr:" + str(r_tr) + " p_tr:" + str(p_tr) + " diff:" + str(diff))
                tp += 1
                diffs.append(diff)
                found = True
                break
        if not found:        
            fp += 1            
    fn = positives - tp
    precision = tp/(tp+fp)
    recall = tp/positives
    f1 = 2*(precision*recall)/(precision+recall)
    ME = np.mean(diffs)
    variance = np.var(diffs)
    SD = np.std(diffs)
    
    return positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD

## -- This is required to match the transition arrays when they differ in dimension 
def delete_false_transitions(x, y, thr):
    limit = min(x.shape[0], y.shape[0])
    x_list = []    # It will contain the false transitions in the x array
    y_list = []    # It will contain the false transitions in the y array
    for i in range(0,limit-1):
        diff = abs(x[i] - y[i])    # Compare two transitions with the same index in the two arrays
        if diff > thr:    # If they differ more than the threshold:
            if abs(x[i+1] - y[i]) < thr:    # Compare the next x transition with the current y -> If < thr:
                x_list.append(i)     # Fill x_list with the transition i, since it's a false transition
                x = np.delete(x,x_list)    # Delete from the x vector the i-th transition
                break    # Leave the cycle
            if abs(x[i] - y[i+1]) < thr:    # Compare the current x transition with the nexy y -> If < thr
                y_list.append(i)    # Fill y_list with the i-th transition
                y = np.delete(y, y_list)    # Delete from the y vector the i-th transition
                break
    
    # It works one one transition per time, thus, to cycle it
    # if the two arrays have different dimensions AND at least a transition has been found
    # in the previous run of the function --> recall the function with the new, "clean" arrays.
    if x.shape[0] != y.shape[0] and (x_list != [] or y_list != []):
        delete_false_transitions(x, y, thr)
    return x, y

## -- Trick to cut decimals in lists
def cut_decimals(list):
    list = [round(n, 3) for n in list]
    return list


################################ --- Data --- ################################ 
'''
subjects_each_fold = [
#[20, 25, 21, 6, 4], # 1
[5, 1, 9, 19, 6],    
[4, 16, 6, 15, 5], # 2
[17, 20, 10, 2, 26], # 3
[26, 9, 17, 14, 6], # 4
[7, 6, 17, 10, 3], # 5
[2, 26, 17, 24, 13], # 6
[19, 3, 7, 15, 24], # 7
[25, 4, 6, 2, 7], # 8
[23, 8, 13, 21, 14], # 9
[9, 1, 21, 24, 3] # 10
]
'''
subjects_each_fold = [
[1], # 1     
[2], # 2
[3], # 3
[5], # 4
[6], # 5
[7], # 6
[8], # 7
[9], # 8
[10], # 9
[11], # 10
[12], # 10    
[13], # 10
[14], # 10    
[15], # 10    
[16], # 10    
[17], # 10 
[18], # 10    
[19], # 10    
[20], # 10    
[21], # 10    
[22], # 10    
[23], # 10    
[26] # 10        
]

model = 'FF6'
fold = 23
subjects_per_fold=1

subjects = []
tolerance = 600

#path = "subjects/min-max/windows_20-del_tr-False-slide-False-digits-4/folds_inter/Report_" + model + "/"
#path = "subjects/min-max/clean/windows_20-del_tr-False-slide-True-digits-3-pace-1/folds_inter/Report_" + model + "/"
#path = "subjects/min-max/clean/windows_20-del_tr-True-slide-False-digits-3/folds_inter/Report_" + model + "/"
#path = "subjects/min-max/clean/windows_200-del_tr-False-slide-True-digits-3-pace-20/folds_inter/Report_" + model + "/"
path = "subjects/bpf-20-450_rect/windows_200-del_tr-False-slide-True-digits-3-pace-20/folds_inter/Report_" + model + "/"

#predicted_suffix = '_predicted_clean_350.csv'
predicted_suffix = '_predicted_clean_500.csv'
#predicted_suffix = '_predicted.csv'

fold_data_path = path + 'Fold_Data/';
if not os.path.exists(fold_data_path):
    os.makedirs(fold_data_path)

in_path_log = path + 'Fold_' + str(fold) + '/'



with open(fold_data_path + 'Model_' + model + '_tol_' + str(tolerance) + '_global.csv', mode='a') as data_global:
    data_log_global = csv.writer(data_global, delimiter=';')
    data_log_global.writerow([' ', 'Mean-Error HS', 'SD HS', 'Mean-Error TO', 'SD TO', '#HS', 'TP HS', 'FP HS', 'FN HS', 'Prec HS', 'Recall HS', 'F1 HS', '#TO' ,'TP TO', 'FP TO', 'FN TO', 'Prec TO', 'Recall TO', 'F1 TO'])
    

for n_fold in range(0,fold):
    subjects = subjects_each_fold[n_fold]
    log = in_path_log + 'fold_' + str(n_fold + 1) + '_log_no_dft.txt'
    old_stdout = sys.stdout
    log_file = open(log,"w")

    positives_tos, MEs_tos, variances_tos, SDs_tos, tp_tos, fn_tos, fp_tos, precision_tos, f1_tos, recall_tos = [], [], [], [], [], [], [], [], [], []
    positives_hss, MEs_hss, variances_hss, SDs_hss, tp_hss, fn_hss, fp_hss, precision_hss, f1_hss, recall_hss = [], [], [], [], [], [], [], [], [], []
    global_means = []
    
    sys.stdout = log_file
    print("MODEL " + str(model) + "  ----  FOLD " + str(fold) + "\n\n\n")
    for subj in subjects:
        in_predicted = path + 'Fold_' + str(n_fold + 1) + '/'
        file_predicted = in_predicted + 's' + str(subj) +  predicted_suffix
        in_real = 'subjects/min-max/clean/baso_real/'
        file_real = in_real + 's' + str(subj) + '_baso_real.csv'


    ## -- Load data
        real = csv_load(file_real, delimiter = ',')
        predicted = csv_load(file_predicted, delimiter = ',')

    ## -- Check transitions, both Toe-Offs and Heel-Strikes
        ref_tos = checkTOs(real)
        ref_tos = np.asarray(ref_tos, dtype = int)
        ref_hss = checkHSs(real)
        ref_hss = np.asarray(ref_hss, dtype = int)
        pred_tos = checkTOs(predicted)
        pred_tos = np.asarray(pred_tos, dtype = int)
        pred_hss = checkHSs(predicted)
        pred_hss = np.asarray(pred_hss, dtype = int)



    ## -- Check for the false transitions
        #ref_tos, pred_tos = delete_false_transitions(ref_tos, pred_tos, 200)
        #ref_hss, pred_hss = delete_false_transitions(ref_hss, pred_hss, 200)

        print("\n\n######### --- SUBJECT " + str(subj) + " --- #########\n\n")
        #print("Reference baso: " + str(ref))
        #print("Predicted baso: " + str(pred))

        count = 0

        print("\neference TOs:" + str(ref_tos))
        print("\nPredicted TOs:" + str(pred_tos))
        print("\nReference HSs:" + str(ref_hss))
        print("\nPredicted HSs:" + str(pred_hss))
        print("\nTolerance: " + str(tolerance))    

        positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD = evaluate(ref_tos, pred_tos, tolerance)
        print("\nTOs:")
        print("Positives: " + str(positives))
        print("True Positives: " + str(tp))
        print("False Positives: " + str(fp))
        print("False Negatives: " + str(fn))
        print("Precision: " + str(precision))
        print("Recall: " + str(recall))
        print("F1: " + str(f1))
        print("Diffs: " + str(diffs))
        print("Mean Error: " + str(ME) + " samples")
        print("Variance: " + str(variance) + " samples")
        print("Standard Deviation: " + str(SD) + " samples")

        ## -- Arrays della Fold da salvare
        positives_tos.append(positives)
        MEs_tos.append(ME)
        variances_tos.append(variance)
        SDs_tos.append(SD)
        tp_tos.append(tp)
        fp_tos.append(fp)
        fn_tos.append(fn)
        precision_tos.append(precision)
        recall_tos.append(recall)
        f1_tos.append(f1)

        positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD = evaluate(ref_hss, pred_hss, tolerance)
        print("\nHSs:")
        print("Positives: " + str(positives))
        print("True Positives: " + str(tp))
        print("False Positives: " + str(fp))
        print("False Negatives: " + str(fn))
        print("Precision: " + str(precision))
        print("Recall: " + str(recall))
        print("F1: " + str(f1))
        print("Diffs: " + str(diffs))
        print("Mean Error: " + str(ME) + " samples")
        print("Variance: " + str(variance) + " samples")
        print("Standard Deviation: " + str(SD) + " samples")

        ## -- Arrays delle Fold da salvare
        positives_hss.append(positives)
        MEs_hss.append(ME)
        variances_hss.append(variance)
        SDs_hss.append(SD)
        tp_hss.append(tp)
        fp_hss.append(fp)
        fn_hss.append(fn)
        precision_hss.append(precision)
        recall_hss.append(recall)
        f1_hss.append(f1)

    sys.stdout = old_stdout
    log_file.close()

    MEs_tos, MEs_hss = cut_decimals(MEs_tos), cut_decimals(MEs_hss)
    variances_tos, variances_hss = cut_decimals(variances_tos), cut_decimals(variances_hss)
    SDs_tos, SDs_hss = cut_decimals(SDs_tos), cut_decimals(SDs_hss)
    precision_tos, precision_hss = cut_decimals(precision_tos), cut_decimals(precision_hss)
    recall_tos, recall_hss = cut_decimals(recall_tos), cut_decimals(recall_hss)
    f1_tos, f1_hss = cut_decimals(f1_tos), cut_decimals(f1_hss)

    import statistics

    mean_val = statistics.mean 

    #with open('../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_' + model +  '/Fold_Data/' + 'Fold_' + str(fold) + '_data_tol_' + str(tolerance) + '.csv', mode='w') as data_log:
    with open(fold_data_path + 'Model_' + model + '_tol_' + str(tolerance) + '.csv', mode='a') as data_log:
        data_log_writer = csv.writer(data_log, delimiter=';')

        header = ['Subject', 'Mean-Error HS', 'SD HS', 'Mean-Error TO', 'SD TO', '#HS', 'TP HS', 'FP HS', 'FN HS', 'Prec HS', 'Recall HS', 'F1 HS', '#TO' ,'TP TO', 'FP TO', 'FN TO', 'Prec TO', 'Recall TO', 'F1 TO']
        number_fold = ['\n\nFOLD ' + str(n_fold + 1) + '\n']   
        data_log_writer.writerow(number_fold)
        data_log_writer.writerow(header)

        for i in range(0, subjects_per_fold):
            data_log_writer.writerow([subjects[i], MEs_hss[i], SDs_hss[i], MEs_tos[i], SDs_tos[i], positives_hss[i], tp_hss[i], fp_hss[i], fn_hss[i], precision_hss[i], recall_hss[i], f1_hss[i], positives_tos[i], tp_tos[i], fp_tos[i], fn_tos[i], precision_tos[i], recall_tos[i], f1_tos[i]])
        
        tp_hss_tot, fp_hss_tot, fn_hss_tot = sum(tp_hss), sum(fp_hss), sum(fn_hss)
        tp_tos_tot, fp_tos_tot, fn_tos_tot = sum(tp_tos), sum(fp_tos), sum(fn_tos)
        positives_hss_tot = sum(positives_hss)
        positives_tos_tot = sum(positives_tos)
        precision_hss_tot = np.round(tp_hss_tot/(tp_hss_tot + fp_hss_tot), decimals = 3)
        recall_hss_tot = np.round(tp_hss_tot/positives_hss_tot, decimals = 3)
        f1_hss_tot = np.round(2*(precision_hss_tot * recall_hss_tot)/(precision_hss_tot + recall_hss_tot), decimals = 3)
        precision_tos_tot = np.round(tp_tos_tot/(tp_tos_tot + fp_tos_tot), decimals = 3)
        recall_tos_tot = np.round(tp_tos_tot/positives_tos_tot, decimals = 3)
        f1_tos_tot = np.round(2*(precision_tos_tot * recall_tos_tot)/(precision_tos_tot + recall_tos_tot), decimals = 3)
        
        data_log_writer.writerow([' '])
        data_log_writer.writerow(['Mean Values', mean_val(MEs_hss), mean_val(SDs_hss), mean_val(MEs_tos), mean_val(SDs_tos), positives_hss_tot, tp_hss_tot, fp_hss_tot, fn_hss_tot, precision_hss_tot, recall_hss_tot, f1_hss_tot, positives_tos_tot, tp_tos_tot, fp_tos_tot, fn_tos_tot, precision_tos_tot, recall_tos_tot, f1_tos_tot])
        data_log_writer.writerow([' '])
        
        with open(fold_data_path + 'Model_' + model + '_tol_' + str(tolerance) + '_global.csv', mode='a') as data_global:
            data_log_global = csv.writer(data_global, delimiter=';')
            data_log_global.writerow(['Fold ' + str(n_fold + 1), mean_val(MEs_hss), mean_val(SDs_hss), mean_val(MEs_tos), mean_val(SDs_tos), positives_hss_tot, tp_hss_tot, fp_hss_tot, fn_hss_tot, precision_hss_tot, recall_hss_tot, f1_hss_tot, positives_tos_tot, tp_tos_tot, fp_tos_tot, fn_tos_tot, precision_tos_tot, recall_tos_tot, f1_tos_tot])

'''
data = pd.read_csv(fold_data_path + 'Model_' + model + '_tol_' + str(tolerance) + '_global.csv', sep = ';')
data.as_matrix
averages = ['Mean Values', mean_val(data['Mean-Error HS']), mean_val(data['SD HS']), mean_val(data['Mean-Error TO']), mean_val(data['SD TO']), 
           mean_val(data['#HS']), mean_val(data['TP HS']), mean_val(data['FP HS']), mean_val(data['FN HS']),
            mean_val(data['Prec HS']), mean_val(data['Recall HS']), mean_val(data['F1 HS']), mean_val(data['#TO']),
            mean_val(data['TP TO']), mean_val(data['FP TO']), mean_val(data['FN TO']), mean_val(data['Prec TO']),
            mean_val(data['Recall TO']), mean_val(data['F1 TO'])
           ]
'''

with open(fold_data_path + 'Model_' + model + '_tol_' + str(tolerance) + '_global.csv', mode='a') as data_global:
    data_log_global = csv.writer(data_global, delimiter=';')
    data_log_global.writerow(['\n'])
    #data_log_global.writerow(averages)

In [2]:
import numpy as np
from numpy import genfromtxt as csv_load
import os
import pandas as pd

# def compute_threshold(v, phase):
in_path = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_FF4/Fold_1/'
file = in_path + 's4_predicted_clean_350.csv'
v = csv_load(file, delimiter = ',')
phase = 1
phase_length = []
count = 0
print(v)
def checkHSs(v):
    hss = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 1 and v[s+1] == 0):
            hss.append(count)
    return hss

def checkTOs(v):
    tos = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 0 and v[s+1] == 1):
            tos.append(count)
    return tos

hss = checkHSs(v)
tos = checkTOs(v)
# hss.insert(0,0)



hss = np.asarray(hss, dtype = int)
tos = np.asarray(tos, dtype = int)

limit = min(len(hss),len(tos))
stances = np.zeros(limit, dtype = int)
swings = np.zeros(limit, dtype = int)


## -- Stance = hss - tos
for i in range(0, limit):
    stances[i] = tos[i] - hss[i]
    
mean_stance = np.mean(stances)
mean_stance = np.around(mean_stance, decimals = 2)
standard_dev_stance = np.std(stances)
standard_dev_stance = np.around(standard_dev_stance, decimals = 2)

print('Mean Stance +/- SD: ')
print(str(mean_stance) + ' +/- ' +  str(standard_dev_stance))

## -- Swing = tos - hss
for j in range(0, limit):
    swings[j] = abs(hss[j + 1] - tos[j])

mean_swing = np.mean(swings)
mean_swing = np.around(mean_swing, decimals = 2)
standard_dev_swing = np.std(swings)
standard_dev_swing = np.around(standard_dev_swing, decimals = 2)

print('Mean Stance +/- SD: ')
print(str(mean_swing) + ' +/- ' +  str(standard_dev_swing))

OSError: ../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_FF4/Fold_1/s4_predicted_clean_350.csv not found.